Implement the MiniMax algorithm with the following scoring function:
Score = MaterialV alue + PositionalV alue
For computing the MaterialValue, each piece is assigned a value (e.g., Pawn = 1, Knight = 3, Bishop = 3, Rook = 5, Queen = 9). Then you sum these values for your pieces and substract the value of the pieces of the oponent.
For computing the PositionalValue, you should take into account the position of each pieces on the board (e.g the more squares a pawn has travelled, the higher their Posi- tionalValue etc.). You should then substract the opponent’s PositionalValue from your pieces’ PositionalValue. 

Firstly I calculate the material value of all the pieces of a given color on the chessboard. I iterate over the board, check the type and color of each piece, and accumulate. their values. After that, I calculate the positional value of all the pieces of a given color on the chessboard. I iterate over the board, check the type and color of each piece, and accumulate their positional values.
Lastly, I calculate an overall evaluation score for a chessboard. I takes into account the material value, positional value, and piece activity value. The final score is a combination of these three factors and is used to determine the quality of the current board position.

In [ ]:
piece_value = {
    'pawn': 1,
    'knight': 3,
    'bishop': 3,
    'rook': 5,
    'queen': 9,
    'king': 0  
}

positional_value = {
    'pawn': [[0, 0, 0, 0, 0, 0, 0, 0],
             [1, 1, 1, 1, 1, 1, 1, 1],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0, 0, 0]],
}
def calculate_material_value(board, color):
    material_value = 0
    for i in range(8):
        for j in range(8):
            piece = board[i][j]
            if isinstance(piece, ChessPiece) and piece.color == color:
                material_value += piece_value[piece.piece_type]
    return material_value

def calculate_positional_value(board, color):
    positional_value = 0
    for i in range(8):
        for j in range(8):
            piece = board[i][j]
            if isinstance(piece, ChessPiece) and piece.color == color:
                positional_value += positional_value[piece.piece_type][i][j]
    return positional_value

def evaluate(board):
    player_color = board.get_player_color()
    opponent_color = board.get_opponent_color()

    material_value = calculate_material_value(board, player_color) - calculate_material_value(board, opponent_color)
    
    positional_value = calculate_positional_value(board, player_color) - calculate_positional_value(board, opponent_color)
    
    piece_activity_value = calculate_piece_activity(board, player_color) - calculate_piece_activity(board, opponent_color)

    final_score = material_value + positional_value + piece_activity_value

    return final_score

Implement Alpha-Beta Prunning.

Alpha-beta pruning is applied in the minimax function, in both two main sections: one for the maximizing player and one for the minimizing player.
It works by initially setting alpha to negative infinity. After that the algorithm explores potential moves, it updates the alpha value, representing the best result achievable for the maximizing player. If the beta value of the current branch becomes less than or equal to the alpha value, it means that the minimizing player has a better option elsewhere, so there's no need to explore further in this branch. The algorithm can break out of the loop, effectively pruning the rest of the branch.

In [ ]:
@log_tree
def minimax(board, depth, alpha, beta, max_player, save_move, data):

    if depth == 0 or board.is_terminal():
        data[1] = board.evaluate()
        return data

    if max_player:
        max_eval = -math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color != board.get_player_color():
                    piece = board[i][j]
                    moves = piece.filter_moves(piece.get_moves(board), board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax(board, depth - 1, alpha, beta, False, False, data)[1]
                        if save_move:
                            if evaluation >= max_eval:
                                if evaluation > data[1]:
                                    data.clear()
                                    data[1] = evaluation
                                    data[0] = [piece, move, evaluation]
                                elif evaluation == data[1]:
                                    data[0].append([piece, move, evaluation])
                        board.unmake_move(piece)
                        max_eval = max(max_eval, evaluation)
                        alpha = max(alpha, evaluation)
                        if beta <= alpha:  # Alpha-beta pruning here
                            break
        return data
    else:
        min_eval = math.inf
        for i in range(8):
            for j in range(8):
                if isinstance(board[i][j], ChessPiece) and board[i][j].color == board.get_player_color():
                    piece = board[i][j]
                    moves = piece.get_moves(board)
                    for move in moves:
                        board.make_move(piece, move[0], move[1], keep_history=True)
                        evaluation = minimax(board, depth - 1, alpha, beta, True, False, data)[1]
                        board.unmake_move(piece)
                        min_eval = min(min_eval, evaluation)
                        beta = min(beta, evaluation)
                        if beta <= alpha:  # Alpha-beta pruning here
                            break
        return data

Implement an improved scoring (evaluation) method for MiniMax. For example, you could add values like KingSafetyValue, MobilityValue (nr of legal moves to each side), PawnStructureValue (can include penalties for isolated pawns, doubled pawns, and bonuses for passed pawns or a strong pawn chain), etc. You can also use Heuristic Evaluation Functions. Be creative!

For this task I made changes to the evaluate function and added a calculate piece activity function in order to improve it. So, basically now calculate_piece_activity function computes a value that represents the activity of a player's pieces, considering the number of legal moves available to each piece. After that, the score is computed by subtracting the material and positional values of the opponent from the values of the player, along with a piece activity value.

In [1]:
def evaluate(board):
    player_color = board.get_player_color()
    opponent_color = board.get_opponent_color()

    material_value = calculate_material_value(board, player_color) - calculate_material_value(board, opponent_color)
    
    positional_value = calculate_positional_value(board, player_color) - calculate_positional_value(board, opponent_color)
    
    piece_activity_value = calculate_piece_activity(board, player_color) - calculate_piece_activity(board, opponent_color)

    final_score = material_value + positional_value + piece_activity_value

    return final_score

def calculate_piece_activity(board, color):
    piece_activity_value = 0
    for i in range(8):
        for j in range(8):
            piece = board[i][j]
            if isinstance(piece, ChessPiece) and piece.color == color:
                moves = piece.filter_moves(piece.get_moves(board), board)
                piece_activity_value += len(moves)
    return piece_activity_value